In [1]:
# All necessary installs (probably don't have to do this anymore)

%pip install transformers datasets torch accelerate
%pip install "accelerate>=0.26.0"
%pip install --upgrade transformers accelerate
%pip install datasets
%pip install ipywidgets
%pip install inseq
!pip install seaborn
# IMPORTANT: Install inseq from source like below
!pip install git+https://github.com/inseq-team/inseq.git@main bitsandbytes

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/cvmfs/hpc.rug.nl/versions/2023.01/rocky8/x86_64/intel/icelake/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/cvmfs/hpc.rug.nl/versions/2023.01/rocky8/x86_64/intel/icelake/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/cvmfs/hpc.rug.nl/versions/2023.01/rocky8/x86_64/intel/icelake/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/cvmfs/hpc.rug.nl/versions/2023.01/rocky8/x86_64/intel/icelake/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/cvmfs/hpc.rug.nl/versions/2023.01/rocky8/x86_64/intel/icelake/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/cvmfs/hpc.rug.nl/versions/2023.01/rocky8/x86_64/intel/icelake/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/cvmfs/hpc.rug.nl/versions/2023.01/rocky8/x86_64/intel/icelake/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/inseq-team/inseq.git (to revision main) to /tmp/pip-req-build-7sz7deyx
  Running command git clone --filter=blob:none --quiet https://github.com/inseq-team/inseq.git /tmp/pip-req-build-7sz7deyx
  Resolved https://github.com/inseq-team/inseq.git to commit 1078e1917834f00139dd369e7eb3e48abfed795c
  Installing build dependencies ... done
  Getting requirements to build 

You should consider upgrading via the '/cvmfs/hpc.rug.nl/versions/2023.01/rocky8/x86_64/intel/icelake/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.


In [ ]:
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from tqdm import tqdm
from inseq.utils.typing import TokenWithId
import torch
import torch.nn.functional as F
import re
import inseq
import pandas as pd


os.environ["TOKENIZERS_PARALLELISM"] = "false"

def load_model(modeltouse):
    os.system("rm -rf ~/.cache/huggingface/hub/*")
    os.system("rm -rf ~/.cache/huggingface/hub/models")
    tokenizer = AutoTokenizer.from_pretrained(modeltouse)
    model = AutoModelForCausalLM.from_pretrained(
        modeltouse,
        quantization_config=BitsAndBytesConfig(load_in_8bit=True),
        device_map="cuda"
    )
    return tokenizer, model

def get_messages(question, question_type, numberofsteps):
    if pd.notna(question):
        if question_type == "Multiple Choice":
            if numberofsteps > 0:
                messages = [
                    {"role": "system", "content": "You are a helpful assistant. Answer the user's multiple choices questions by reasoning step-by-step. Your answer must be a single letter (A, B, C, or D). Only answer with one of these letters."},
                    {"role": "user", "content": question},
                ]
            else:
                messages = [
                    {"role": "system", "content": "You are a helpful assistant. Answer the user's multiple choices questions. Your answer must be a single letter (A, B, C, or D)."},
                    {"role": "user", "content": question},
                ]
        elif question_type == "Open Ended":
            if numberofsteps > 0:
                messages = [
                    {"role": "system", "content": "You are a helpful assistant. Answer the user questions by reasoning step-by-step. Answer in max 10 words."},
                    {"role": "user", "content": question},
                ]
            else:
                messages = [
                    {"role": "system", "content": "You are a helpful assistant. Answer the user questions without providing any reasoning or chain of thought steps.  Answer in max 10 words."},
                    {"role": "user", "content": question},
                ]
        elif question_type == "Math":
            if numberofsteps > 0:
                messages = [
                    {"role": "system", "content": "You are a helpful assistant. Answer the user questions by reasoning step-by-step. After reasoning, output only the final number as your answer, without explanation.  Answer in max 10 words "},
                    {"role": "user", "content": question},
                ]
            else:
                messages = [
                    {"role": "system", "content": "You are a helpful assistant.  Answer the user questions without providing any reasoning or chain of thought steps. Output only the number corresponding to the answer, without explanation.  Answer in max 10 words"},
                    {"role": "user", "content": question},
                ]
        elif question_type == "T/F":
            if numberofsteps > 0:
                messages = [
                    {"role": "system", "content": "You are a helpful assistant. Answer the user questions by reasoning step-by-step. Your final answer must be 'TRUE' or 'FALSE'."},
                    {"role": "user", "content": question},
                ]
            else:
                messages = [
                    {"role": "system", "content": "You are a helpful assistant. Answer the user question in one word: 'TRUE' or 'FALSE'."},
                    {"role": "user", "content": question},
                ]
        return messages
    return []

def get_max_tokens(question_type):
    if question_type == "Multiple Choice":
        return 1 # Allow for only 1 token
    elif question_type == "Open Ended":
        return 500  # Allow up to 500 tokens
    elif question_type == "Math":
        return 500  # Allow up to 500 tokens
    elif question_type == "T/F":
        return 1 # Allow for only 1 token

def calc_confidence(inputs, outputs):
    score_list = outputs.scores
    input_length = inputs["input_ids"].shape[1]
    log_probs = []
    for i, logits in enumerate(score_list):
        probs = F.softmax(logits, dim=-1)
        token_id = outputs.sequences[0][input_length + i]
        token_prob = probs[0, token_id]
        log_probs.append(torch.log(token_prob))
    if log_probs:
        total_log_prob = sum(log_probs)
        confidence = torch.exp(total_log_prob)
    else:
        confidence = torch.tensor(0.0)
    return confidence

def output_and_attribute(messages, max_new_tokens, tokenizer, model):
    input_text = "\n".join([msg["content"] for msg in messages])
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs["input_ids"],
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        output_scores=True,
        return_dict_in_generate=True
    )
    decoded_output = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    if "So the answer is:" in decoded_output:
        final_answer = decoded_output.split("So the answer is:")[-1].strip()
    elif "The answer to the question is:" in decoded_output:
        final_answer = decoded_output.split("The answer to the question is:")[-1].strip()
    else:
        final_answer = decoded_output.strip()
    confidence = calc_confidence(inputs, outputs)
    confidence_val = confidence.item() if isinstance(confidence, torch.Tensor) else confidence
    num_generated_tokens = outputs.sequences[0].shape[0] - inputs["input_ids"].shape[1]
    return final_answer, confidence_val, num_generated_tokens

def find_most_salient_step(messages, row, n_steps, tokenizer, model, question_type):
    # Build a full message with all reasoning steps
    full_messages = messages.copy()
    reasoning_steps = "Let's think step by step:\n"
    for i in range(1, n_steps + 1):
        step_content = row[f"Step_{i}"]
        if pd.notna(step_content):
            reasoning_steps += f"~Step {i}: {step_content}\n"
    full_messages.append({"role": "assistant", "content": reasoning_steps + " So the answer is:"})
    chat_str = tokenizer.apply_chat_template(full_messages, tokenize=False, continue_final_message=True)
    tokenizer.pad_token = tokenizer.eos_token
    inseq_model = inseq.load_model(model, "integrated_gradients", tokenizer=tokenizer) # Change to "saliency" for saliency
    max_new_tokens = get_max_tokens(question_type)
    attr = inseq_model.attribute(
        chat_str,
        generation_args={
            "do_sample": False,
            "temperature": None,
            "top_k": None,
            "top_p": None,
            "max_new_tokens": max_new_tokens,
            "skip_special_tokens": False
        },
        step_scores=["probability"],
    )[0]
    # Clean tokens
    replace_dict = {"Ċ": "\n", "Ġ": " "}
    def clean_tokens(tokens, d):
        for t in tokens:
            for old, new in d.items():
                t.token = t.token.replace(old, new)
        return tokens
    attr.source = clean_tokens(attr.source, replace_dict)
    attr.target = clean_tokens(attr.target, replace_dict)
    aggregated = attr.aggregate("split", split_pattern="(<\\|im_start\\|>system|<\\|im_start\\|>user|<\\|im_start\\|>assistant|~Step \\d:|So the answer is:)")
    scores = torch.norm(aggregated.target_attributions, p=2, dim=2)
    best_score = 0
    best_content_index = 0
    for i in range(1, n_steps + 1):
        content_index = 6 + 2 * i  
        if content_index < scores.shape[0]:
            score = scores[content_index].item()
            if score > best_score:
                best_score = score
                best_content_index = content_index
    most_salient_step = (best_content_index - 5) // 2
    return most_salient_step

def process_single_question(row, tokenizer, model):
    question = row['Question ']
    question_type = row['Question_Type']
    correct_answer = row['Correct_Answer']
    numberofsteps = sum(pd.notna(row[f"Step_{i}"]) for i in range(1, 6))
    
    # Prepare a base message (without reasoning steps)
    base_messages = get_messages(question, question_type, numberofsteps)
    
    # For the normal configuration: use all steps
    reasoning_normal = ""
    for i in range(1, numberofsteps + 1):
        step_content = row[f"Step_{i}"]
        if pd.notna(step_content):
            reasoning_normal += f"~Step {i}: {step_content}\n"
    messages_normal = base_messages.copy()
    if reasoning_normal:
        messages_normal.append({"role": "assistant", "content": "Let's think step by step:\n" + reasoning_normal + " So the answer is:"})
    else:
        messages_normal.append({"role": "assistant", "content": " The answer to the question is:"})
    max_tokens_normal = get_max_tokens(question_type)
    answer_normal, conf_normal, tokens_normal = output_and_attribute(messages_normal, max_tokens_normal, tokenizer, model)
    
    # For salient removal configuration: remove the most salient step
    # First, if available, determine the most salient step from the normal run
    most_salient_step = find_most_salient_step(base_messages, row, numberofsteps, tokenizer, model, question_type) if numberofsteps > 0 else None
    reasoning_salient = ""
    if most_salient_step is not None:
        for i in range(1, numberofsteps + 1):
            if i == most_salient_step:
                continue
            step_content = row[f"Step_{i}"]
            if pd.notna(step_content):
                reasoning_salient += f"~Step {i}: {step_content}\n"
    messages_salient = base_messages.copy()
    if reasoning_salient:
        messages_salient.append({"role": "assistant", "content": "Let's think step by step:\n" + reasoning_salient + " So the answer is:"})
    else:
        messages_salient.append({"role": "assistant", "content": " The answer to the question is:"})
    max_tokens_salient = get_max_tokens(question_type)
    answer_salient, conf_salient, tokens_salient = output_and_attribute(messages_salient, max_tokens_salient, tokenizer, model)
    
    # For no_steps configuration: remove all reasoning steps
    messages_no_steps = base_messages.copy()
    messages_no_steps.append({"role": "assistant", "content": " The answer to the question is:"})
    max_tokens_no_steps = get_max_tokens(question_type)
    answer_no_steps, conf_no_steps, tokens_no_steps = output_and_attribute(messages_no_steps, max_tokens_no_steps, tokenizer, model)
    
    # Build a single result row combining the three configurations
    result = {
        "Question": question,
        "Question_Type": question_type,
        "Correct_Answer": correct_answer,
        "Number_of_Steps": numberofsteps,
        "Most_Salient_Step": most_salient_step,
        "Answer_normal": answer_normal,
        "Confidence_normal": conf_normal,
        "Number_of_Output_Tokens_normal": tokens_normal,
        "Answer_after_removal": answer_salient,
        "Confidence_after_removal": conf_salient,
        "Number_of_Output_Tokens_after_removal": tokens_salient,
        "Answer_no_steps": answer_no_steps,
        "Confidence_no_steps": conf_no_steps,
        "Number_of_Output_Tokens_no_steps": tokens_no_steps
    }
    return result

def run_experiment(file, range_input, model_loaded="no", modeltouse="Qwen/Qwen2.5-1.5B-Instruct"):
    # Load CSV and filter for valid questions
    df = pd.read_csv(file)
    df_filtered = df[df['Question '].notna()]
    try:
        start, end = map(int, range_input.split('-'))
    except Exception as e:
        print("Error with your indexing. Please specify the range_input as n-n.")
        return

    # Load model if needed
    if model_loaded == "no":
        tokenizer, model = load_model(modeltouse)
    else:
        # Assume model and tokenizer are already loaded
        tokenizer, model = None, None  # You can adjust this if needed
    
    # Process each question individually and collect results
    all_results = []
    for idx, row in tqdm(df_filtered.iloc[start:end+1].iterrows(), total=(end-start+1), desc="Processing Questions", unit="question"):
        result_row = process_single_question(row, tokenizer, model)
        all_results.append(result_row)
    
    results_df = pd.DataFrame(all_results)
    results_df.to_csv('experiment_results.csv', index=False)
    print("Results saved to 'experiment_results.csv'")

# Example call to run the experiment
if __name__ == "__main__":
    run_experiment(
        file="Our_dataset_final.csv", 
        range_input="60-60", # Adjust this range as needed
        model_loaded="no", # Set to "no" to load the model, or "yes" if already loaded
        modeltouse="Qwen/Qwen2.5-1.5B-Instruct" # Load the model 
    )


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Processing Questions:   0%|          | 0/1 [00:00<?, ?question/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The model is loaded in 8bit mode. The device cannot be changed after loading the model.

Processing Questions: 100%|██████████| 1/1 [00:18<00:00, 18.15s/question]

Results saved to 'experiment_results.csv'
